# imports

In [1]:
import mysql.connector
import json
import pandas as pd
from tqdm.notebook import tqdm
from datetime import datetime
import re
import numpy as np
import bcrypt

# connect to db

In [2]:
with open("../crawling/tokens.json") as f:
    passwords = json.load(f)

pwd=passwords["mysql_password_laptop"]
# pwd=passwords["mysql_password"]

In [3]:
# Connect to MySQL
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password=pwd,
    database='stampcollection'
)

cursor = connection.cursor()

# create salted hash password

In [6]:
# Hash a password
def hash_password(password):
    salt = bcrypt.gensalt()
    hashed = bcrypt.hashpw(password.encode('utf-8'), salt)
    return hashed

# Verify a password
def verify_password(password, hashed):
    return bcrypt.checkpw(password.encode('utf-8'), hashed)

# Example usage
password = "secure_password"
hashed_password = hash_password(password)
print("Stored hash:", hashed_password)

# Verification
is_valid = verify_password("secure_password", hashed_password)
print("Password valid?", is_valid)

Stored hash: b'$2b$12$TUspeYFK1R32Y6P8HZgdd.Ja/OWfmxJfV5l6m5jPlJI8i0xY.wlHG'
Password valid? True


## add user

In [9]:
# Add user to the database
def add_user(username, email, password):
    hashed_password = hash_password(password)
    query = """
        INSERT INTO users (username, email, password_hash)
        VALUES (%s, %s, %s)
    """
    cursor.execute(query, (username, email, hashed_password))
    connection.commit()
    print(f"User '{username}' added successfully!")

In [12]:
# example
cursor.execute("SET FOREIGN_KEY_CHECKS = 0")
cursor.execute("TRUNCATE TABLE users")
cursor.execute("SET FOREIGN_KEY_CHECKS = 1")
connection.commit()
add_user("example_user", "user@example.com", "secure_password")

User 'example_user' added successfully!


# add stamp to user

In [5]:
# Function to add a user-stamp ownership
def add_or_update_user_stamp(user_id, stamp_id, amount_used=1, note=None):
    query = """
        INSERT INTO user_stamps (user_id, stamp_id, amount_used, note)
        VALUES (%s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE 
            amount_used = amount_used + VALUES(amount_used)
    """
    cursor.execute(query, (user_id, stamp_id, amount_used, note))
    connection.commit()
    print(f"Updated or added stamp {stamp_id} for user {user_id}.")

# Retrieve the user_id for example_user
def get_user_id(username):
    query = "SELECT user_id FROM users WHERE username = %s"
    cursor.execute(query, (username,))
    result = cursor.fetchone()
    if result:
        return result[0]
    else:
        raise ValueError(f"User '{username}' not found.")

In [6]:
# Example usage
try:
    # cursor.execute("SET FOREIGN_KEY_CHECKS = 0")
    # cursor.execute("TRUNCATE TABLE user_stamps")
    # cursor.execute("SET FOREIGN_KEY_CHECKS = 1")
    example_user_id = get_user_id("Timo1024")
    add_or_update_user_stamp(example_user_id, stamp_id=4578, amount_used=1)
    add_or_update_user_stamp(example_user_id, stamp_id=579, amount_used=1)
    add_or_update_user_stamp(example_user_id, stamp_id=345, amount_used=1)
    add_or_update_user_stamp(example_user_id, stamp_id=69, amount_used=1)
    add_or_update_user_stamp(example_user_id, stamp_id=420, amount_used=1)
    add_or_update_user_stamp(example_user_id, stamp_id=2, amount_used=1)
    add_or_update_user_stamp(example_user_id, stamp_id=54678, amount_used=1)
    add_or_update_user_stamp(example_user_id, stamp_id=4353, amount_used=1)
    add_or_update_user_stamp(example_user_id, stamp_id=906, amount_used=1)
    add_or_update_user_stamp(example_user_id, stamp_id=2510, amount_used=1)
    add_or_update_user_stamp(example_user_id, stamp_id=609, amount_used=1)
except ValueError as e:
    print(e)

Updated or added stamp 4578 for user 1.
Updated or added stamp 579 for user 1.
Updated or added stamp 345 for user 1.
Updated or added stamp 69 for user 1.
Updated or added stamp 420 for user 1.
Updated or added stamp 2 for user 1.
Updated or added stamp 54678 for user 1.
Updated or added stamp 4353 for user 1.
Updated or added stamp 906 for user 1.
Updated or added stamp 2510 for user 1.
Updated or added stamp 609 for user 1.
